https://discourse.pymc.io/t/a-bayesian-approach-to-media-mix-modeling-by-michael-johns-zhenyu-wang/6024

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

predictors = weekly marketing spend in each channel, transformed
* non linear functional transformation to account for diminishing returns on spend
* adstock tranformation of channel spend to account for lagged effects of advertising

Reach function: non linear shape function with a single parameter mu to capture diminishing returns on spend
<br>mu controls how quickly the curve will saturate. mu = 1 gives a linear curve
$$
[\frac{1 - e^{-\mu x_mt}}{1 + e^{-\mu x_mt}}]
$$


Adstock to capture carry over effects of marketing: allow marketing signal to decay over time<br>
This is a geometric decay. <br>
As alpha goes up, the decay rate slows down and the impact of the marketing is assumed over a longer period of time

$$
[\frac{\sum_{l=0}^{t-1} \alpha^l x_{t-l}}{\sum_{l=0}^{t-1}\alpha^l}\
$$

Bayesian methods can
1. set constraints on model parameters
2. incorporate existing knowledge using priors
3. help learn saturation and decay parameters. as a part of model fitting
4. produce a generative model that is well suited simulation

Base model likelihood and prior structure
$$
\begin{equation}
y_t = \alpha + \sum_{m=1}^{M} \beta_m f(x_{t,m}) + \sum_{c=1}^c \beta_cZ_{t,c} + e_t
\end{equation}
$$
where

$y_t$ = new customers expected value \
$\beta_m$ = is the marketing channel that is instantiated with a prior, m is the number of marketing channels \
$\beta_c$ = the beta for the control variable  \
$\alpha$ = a prior for the alpha parameter for the adstock transformation we apply to a subset for our channels. \
$mu$ = A prior for mu that controls the saturation curve for each channel. \
$e_t$ = prior for our noise term

In [8]:
import pymc3 as pm

def saturation_function(x_t, mu):
  return (1 - np.exp(-mu * x_t))/(1 + np.exp(-mu * x_t))

with pm.Model() as example_model:
  intercept = pm.Normal("intercept", 0, sigma=1)
  beta = pm.HalfNormal("beta", sd=1, shape=3)
  half_sat = pm.Gamma("half_sat", alpha=3, beta=1, shape=3)
  alpha = pm.Beta("alpha", alpha=1, beta=3)
  c_beta = pm.Normal("control_beta", sd=1)
  sigma = pm.Exponential("sigma", 10)

  mu = (
      intercept + 
      beta[0] * saturation_function(500, half_sat[0]) +
      beta[1] * saturation_function(500, half_sat[1]) +
      beta[2] * saturation_function(100, half_sat[2]) +
      c_beta * Z1
  )

  y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=data)
  trace = sample()

AttributeError: module 'arviz' has no attribute 'geweke'